In [1]:
import pandas as pd
from scipy.stats.stats import pearsonr
import statsmodels.api as sm
import numpy as np

# Table of Contents

Below analyses are for both individual and small group markets. These links do not work within the GitHub viewer, but they do work inside Jupyter Notebok.

### Within Year Analysis
- [Mean, Variance, Min, Max, etc.](#a1)
- [Correlation Coefficients Within Year](#a2)

Linear regressions for the following:

- [Costs 2016 ~ Premiums 2016 + Transfers 2016](#b6)
- [Costs 2015 ~ Premiums 2015 + Transfers 2015](#b7)
- [Costs 2014 ~ Premiums 2014 + Transfers 2014](#b8)

And if Profits_0 = Premium - Costs, linear regressions for the following:

- [Profits_1 2016 ~ Transfers 2016](#b9a)
- [Profits_1 2015 ~ Transfers 2015](#b9b)
- [Profits_1 2014 ~ Transfers 2014](#b9c)

And if Profits_1 = Premium - Costs + Reinsurance, to investigate the impacts of introducing reinsurance in the individual group, linear regressions for the following in the individual group only:

- [Profits_1 2016 ~ Transfers 2016](#b10a)
- [Profits_1 2015 ~ Transfers 2015](#b10b)
- [Profits_1 2014 ~ Transfers 2014](#b10c)

### Between Year Analysis
If Profits_2 = Premiums - Costs + Transfers + Reinsurance, linear regressions for the following:

- [Transfers 2016 ~ Profits_2 2014 + Transfers 2015](#b1)
- [Transfers 2016 ~ Profits_2 2014 + Transfers 2014](#b2)
- [Transfers 2016 ~ Profits_2 2014 + Transfers 2014 + Loss 2015 + Transfers 2015](#b3)
- [Transfers 2015 ~ Profits_2 2014 + Transfers 2014](#b4)
- [Transfers 2016 ~ Profits_2 2015 + Transfers 2015](#b5)

### [Impact on Exiting the Market](#c1)
Logistic regressions for the following:

- [ExitedMarketIn2016 ~ Q1 Profits_2 2014 + Q5 Profits_2 2015](#c2)
- [ExitedMarketIn2016 ~ NormalizedProfits_2 2014 + NormalizedProfits_2 2015](#c3)


#  Cleaning the data

- Import the data

- Create the labels

- Filter the data to only include fields we want

- Drop companies with <2000 MM

- Calculate transfers, premiums, costs, and reinsurance (if individual group) per member month

In [2]:
data2014 = pd.read_csv("Our_Dataset/2014dataset.csv")
data2015 = pd.read_csv("Our_Dataset/2015dataset.csv")
data2016 = pd.read_csv("Our_Dataset/2016dataset.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,52,323,649,882,932,1092,1202) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,57,204,341,376,377,396,430,431,928,944,1006,1013,1151,1160,1169,1179,1411) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,17,163,355,406,702,835,847,1083,1156,1183,1185,1200) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Create the labels.
# By index:
# label 2 is transfers
# label 3 is premiums
# label 4 is costs
# label 5 is member months
# label 6 is reinsurance (individual group only)

ind_labels_14 = ["HIOS ID", "HIOS INPUTTED INSURANCE COMPANY NAME",
                  "HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)",
                  "1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15",
                  "2.16Totalincurredclaims2HealthInsuranceINDIVIDUALTotalasof3/31/15",
                  "7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15",
                  "REINSURANCE PAYMENT AMOUNT (OR NOT ELIGIBLE)"]
ind_labels_15 = [label.replace('15', '16') for label in ind_labels_14]
ind_labels_16 = [label.replace('15', '17') for label in ind_labels_14]

sg_labels_14 = ["HIOS ID", "HIOS INPUTTED INSURANCE COMPANY NAME",
                 "HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP MARKET)",
                  "1.1Directpremiumwritten7HealthInsuranceSMALLGROUPTotalasof3/31/15",
                  "2.16Totalincurredclaims7HealthInsuranceSMALLGROUPTotalasof3/31/15",
                  "7.4Membermonths7HealthInsuranceSMALLGROUPTotalasof3/31/15"]
sg_labels_15 = [label.replace('15', '16') for label in sg_labels_14]
sg_labels_16 = [label.replace('15', '17') for label in sg_labels_14]

In [4]:
# Filter the data to include only the fields we want

ind_data_14 = data2014[ind_labels_14].dropna()
ind_data_15 = data2015[ind_labels_15].dropna()
ind_data_16 = data2016[ind_labels_16].dropna()
sg_data_14 = data2014[sg_labels_14].dropna()
sg_data_15 = data2015[sg_labels_15].dropna()
sg_data_16 = data2016[sg_labels_16].dropna()

In [5]:
# Drop companies with <2000 MM
ind_data_14 = ind_data_14.where(ind_data_14[ind_labels_14[5]] > 2000).dropna()
ind_data_15 = ind_data_15.where(ind_data_15[ind_labels_15[5]] > 2000).dropna()
ind_data_16 = ind_data_16.where(ind_data_16[ind_labels_16[5]] > 2000).dropna()
sg_data_14 = sg_data_14.where(sg_data_14[sg_labels_14[5]] > 2000).dropna()
sg_data_15 = sg_data_15.where(sg_data_15[sg_labels_15[5]] > 2000).dropna()
sg_data_16 = sg_data_16.where(sg_data_16[sg_labels_16[5]] > 2000).dropna()

In [6]:
# Calculate transfers, premiums, costs, and reinsurance (if individual group) per member month
def calculate_pmm(df, df_labels, group):
    df["MM"] = df[df_labels[5]]
    df["TransfersPMM"] = df[df_labels[2]] / df[df_labels[5]]
    df["PremiumsPMM"] = abs(df[df_labels[3]] / df[df_labels[5]])
    df["CostsPMM"] = abs(df[df_labels[4]] / df[df_labels[5]])
    if group == "i":
        df["ReinsurancePMM"] = df[df_labels[6]] / df[df_labels[5]]
        df["ProfitsPMM"] = df["PremiumsPMM"] - df["CostsPMM"] + df["TransfersPMM"] + df["ReinsurancePMM"]
        df["PremMinusCostsPlusReinsurancePMM"] = df["PremiumsPMM"] - df["CostsPMM"] + df["ReinsurancePMM"]
    elif group == "sg":
        df["ProfitsPMM"] = df["PremiumsPMM"] - df["CostsPMM"] + df["TransfersPMM"]
    df["PremMinusCostsPMM"] = df["PremiumsPMM"] - df["CostsPMM"]
    df["PremMinusCostsPlusTransfersPMM"] = df["PremiumsPMM"] - df["CostsPMM"] + df["TransfersPMM"]
    return df

ind_data_14 = calculate_pmm(ind_data_14, ind_labels_14, "i")
ind_data_15 = calculate_pmm(ind_data_15, ind_labels_15, "i")
ind_data_16 = calculate_pmm(ind_data_16, ind_labels_16, "i")
sg_data_14 = calculate_pmm(sg_data_14, sg_labels_14, "sg")
sg_data_15 = calculate_pmm(sg_data_15, sg_labels_15, "sg")
sg_data_16 = calculate_pmm(sg_data_16, sg_labels_16, "sg")

In [7]:
# Combine the dataframes by left merging

ind_data = ind_data_14.merge(ind_data_15, on='HIOS ID', how='outer', suffixes=('_14', '_15'))\
.merge(ind_data_16, on='HIOS ID', how='outer')

In [8]:
# Do the same thing for small group

sg_data = sg_data_14.merge(sg_data_15, on='HIOS ID', how='outer', suffixes=('_14', '_15'))\
.merge(sg_data_16, on='HIOS ID', how='outer')

# Within-year analysis

- Mean, Variance, Min, Max

- Correlation coefficient analysis within each year

<a id='a1'></a>
### Mean, Variance, Min, Max

In [9]:
# Mean, Variance, Min, Max in a nice dataframe

df = pd.DataFrame(columns=["Year", "Group", "N", "Avg Costs", "Std Costs",
                           "Avg Premiums", "Std Premiums",
                           "Avg Abs Transfers", "Std Abs Transfers",
                           "Avg Transfers", "Std Transfers",
                           "Weighted Avg Transfers",
                           "Avg Profits", "Std Profits",
                           "Avg Reinsurance", "Std Reinsurance"])

years = [2014, 2015, 2016]
groups = ["Individual", "Small Group"]
ind_dfs = dict()
ind_dfs[2014] = ind_data_14
ind_dfs[2015] = ind_data_15
ind_dfs[2016] = ind_data_16
sg_dfs = dict()
sg_dfs[2014] = sg_data_14
sg_dfs[2015] = sg_data_15
sg_dfs[2016] = sg_data_16
data = dict()
data["Individual"] = ind_dfs
data["Small Group"] = sg_dfs
labels = ["CostsPMM", "PremiumsPMM", "TransfersPMM", "ProfitsPMM", "ReinsurancePMM"]

i = 0
for year in years:
    for group in groups:
        b = labels
        a = data[group][year]
        if group == "Individual":
            df.loc[i] = [year, group, len(a),
                         round(np.mean(a[b[0]])), round(np.std(a[b[0]])),
                         round(np.mean(a[b[1]])), round(np.std(a[b[1]])),
                         round(np.mean(abs(a[b[2]]))), round(np.std(abs(a[b[2]]))),
                         round(np.mean(a[b[2]])), round(np.std(a[b[2]])),
                         round(np.average(a[b[2]], weights=a["MM"])),
                         round(np.mean(a[b[3]])), round(np.std(a[b[3]])),
                         round(np.mean(a[b[4]])), round(np.std(a[b[4]]))]
        else:
            df.loc[i] = [year, group, len(a),
                         round(np.mean(a[b[0]])), round(np.std(a[b[0]])),
                         round(np.mean(a[b[1]])), round(np.std(a[b[1]])),
                         round(np.mean(abs(a[b[2]]))), round(np.std(abs(a[b[2]]))),
                         round(np.mean(a[b[2]])), round(np.std(a[b[2]])),
                         round(np.average(a[b[2]], weights=a["MM"])),
                         round(np.mean(a[b[3]])), round(np.std(a[b[3]])),
                         np.nan, np.nan]
        i = i + 1
df

,Year,Group,N,Avg Costs,Std Costs,Avg Premiums,Std Premiums,Avg Abs Transfers,Std Abs Transfers,Avg Transfers,Std Transfers,Weighted Avg Transfers,Avg Profits,Std Profits,Avg Reinsurance,Std Reinsurance
0,2014,Individual,403,366,183,314,92,42,57,16,69,-0.0,23,83,59,56
1,2014,Small Group,539,304,78,386,81,15,26,-7,29,-0.0,76,56,NaN,NaN
2,2015,Individual,453,392,188,345,78,52,69,8,86,0.0,4,93,43,35
3,2015,Small Group,542,319,83,405,73,28,38,-13,45,0.0,73,76,NaN,NaN
4,2016,Individual,420,416,230,376,79,63,99,9,117,-0.0,-7,99,24,24
5,2016,Small Group,475,340,84,422,70,29,39,-13,47,0.0,69,82,NaN,NaN


<a id='a2'></a>
### Correlation Coefficients within a year

In [10]:
# Correlation coefficient analysis within each year


df2 = pd.DataFrame(columns=["Year", "Group", "N", "Premium vs Transfers r", "Premium vs Transfers p",
                            "Transfers vs Costs r", "Transfers vs Costs p",
                            "Costs vs Premium r", "Costs vs Premium p",
                            "Profits vs Transfers r", "Profits vs Transfers p"])

def within_year_analysis(df, year, group):
    r1 = pearsonr(df["PremiumsPMM"], df["TransfersPMM"])
    r2 = pearsonr(df["TransfersPMM"], df["CostsPMM"])
    r3 = pearsonr(df["CostsPMM"], df["PremiumsPMM"])
    r4 = pearsonr(df["ProfitsPMM"], df["TransfersPMM"])
    return [year, group, len(df), round(r1[0], 4), round(r1[1], 4),
            round(r2[0], 4), round(r2[1], 4), round(r3[0], 4),
            round(r3[1], 4), round(r4[0], 4), round(r4[1], 4)]
    
df2.loc[0] = within_year_analysis(ind_data_14, 2014, "individual")
df2.loc[1] = within_year_analysis(ind_data_15, 2015, "individual")
df2.loc[2] = within_year_analysis(ind_data_16, 2016, "individual")
df2.loc[3] = within_year_analysis(sg_data_14, 2014, "small group")
df2.loc[4] = within_year_analysis(sg_data_15, 2015, "small group")
df2.loc[5] = within_year_analysis(sg_data_16, 2016, "small group")

df2

,Year,Group,N,Premium vs Transfers r,Premium vs Transfers p,Transfers vs Costs r,Transfers vs Costs p,Costs vs Premium r,Costs vs Premium p,Profits vs Transfers r,Profits vs Transfers p
0,2014,individual,403,0.2898,0.0000,0.6666,0.0000,0.6300,0.0,0.0711,0.1542
1,2015,individual,453,0.3993,0.0000,0.7196,0.0000,0.6316,0.0,0.0435,0.3560
2,2016,individual,420,0.4221,0.0000,0.8319,0.0000,0.6372,0.0,-0.2219,0.0000
3,2014,small group,539,0.0098,0.8212,0.1524,0.0004,0.7664,0.0,0.3175,0.0000
4,2015,small group,542,0.1138,0.0080,0.1947,0.0000,0.6443,0.0,0.4942,0.0000
5,2016,small group,475,0.2429,0.0000,0.2509,0.0000,0.5983,0.0,0.5251,0.0000


# Between-year analysis

<a id='b1'></a>
### Transfers 2016 ~ Loss 2014 + Transfers 2014

In [11]:
# INDIVIDUAL GROUP Transfers 2016 = Loss 2014 + Transfers 2014

no_nulls = ind_data[['ProfitsPMM_14', 'TransfersPMM_14', 'TransfersPMM']].dropna()
X = no_nulls[['ProfitsPMM_14', 'TransfersPMM_14']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.391
Model:                            OLS   Adj. R-squared:                  0.386
Method:                 Least Squares   F-statistic:                     92.94
Date:                Wed, 29 May 2019   Prob (F-statistic):           6.47e-32
Time:                        17:27:57   Log-Likelihood:                -1744.9
No. Observations:                 293   AIC:                             3496.
Df Residuals:                     290   BIC:                             3507.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               3.0379      6.070      0.501      0.617      -8.908      14.984
ProfitsPMM_14      -0.2083      0.079     -2.644      0.009      -0.363      -0.053
TransfersPMM_14     1.0920      0.080     13.634      0.000       0.934       1.250
==============================================================================
Omnibus:                      184.317   Durbin-Watson:                   2.085
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3133.297
Skew:                           2.213   Prob(JB):                         0.00
Kurtosis:                      18.397   Cond. No.                         93.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
# SMALL GROUP Transfers 2016 = Loss 2014 + Transfers 2014

no_nulls = sg_data[['ProfitsPMM_14', 'TransfersPMM_14', 'TransfersPMM']].dropna()
X = no_nulls[['ProfitsPMM_14', 'TransfersPMM_14']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.276
Model:                            OLS   Adj. R-squared:                  0.273
Method:                 Least Squares   F-statistic:                     81.88
Date:                Wed, 29 May 2019   Prob (F-statistic):           7.56e-31
Time:                        17:27:57   Log-Likelihood:                -2157.7
No. Observations:                 432   AIC:                             4321.
Df Residuals:                     429   BIC:                             4334.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -4.3722      3.451     -1.267      0.206     -11.156       2.411
ProfitsPMM_14       0.0003      0.036      0.010      0.992      -0.070       0.070
TransfersPMM_14     0.7471      0.062     11.986      0.000       0.625       0.870
==============================================================================
Omnibus:                      237.448   Durbin-Watson:                   1.967
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4739.555
Skew:                          -1.899   Prob(JB):                         0.00
Kurtosis:                      18.776   Cond. No.                         189.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<a id='b2'></a>
### Transfers 2016 ~ Loss 2014 + Transfers 2015

In [13]:
# INDIVIDUAL GROUP Transfers 2016 = Loss 2014 + Transfers 2015

no_nulls = ind_data[['ProfitsPMM_14', 'TransfersPMM_15', 'TransfersPMM']].dropna()
X = no_nulls[['ProfitsPMM_14', 'TransfersPMM_15']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.792
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     546.9
Date:                Wed, 29 May 2019   Prob (F-statistic):           1.25e-98
Time:                        17:27:57   Log-Likelihood:                -1567.5
No. Observations:                 290   AIC:                             3141.
Df Residuals:                     287   BIC:                             3152.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               2.6661      3.515      0.758      0.449      -4.253       9.585
ProfitsPMM_14      -0.1604      0.048     -3.375      0.001      -0.254      -0.067
TransfersPMM_15     1.1779      0.036     32.998      0.000       1.108       1.248
==============================================================================
Omnibus:                       68.662   Durbin-Watson:                   2.047
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              774.635
Skew:                           0.555   Prob(JB):                    6.17e-169
Kurtosis:                      10.929   Cond. No.                         100.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
# SMALL GROUP Transfers 2016 = Loss 2014 + Transfers 2015

no_nulls = sg_data[['ProfitsPMM_14', 'TransfersPMM_15', 'TransfersPMM']].dropna()
X = no_nulls[['ProfitsPMM_14', 'TransfersPMM_15']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.451
Model:                            OLS   Adj. R-squared:                  0.448
Method:                 Least Squares   F-statistic:                     175.8
Date:                Wed, 29 May 2019   Prob (F-statistic):           1.87e-56
Time:                        17:27:57   Log-Likelihood:                -2093.7
No. Observations:                 431   AIC:                             4193.
Df Residuals:                     428   BIC:                             4206.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -6.2944      2.876     -2.189      0.029     -11.947      -0.642
ProfitsPMM_14       0.0290      0.030      0.971      0.332      -0.030       0.088
TransfersPMM_15     0.7650      0.042     18.032      0.000       0.682       0.848
==============================================================================
Omnibus:                      334.162   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14591.083
Skew:                          -2.831   Prob(JB):                         0.00
Kurtosis:                      30.936   Cond. No.                         181.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<a id='b3'></a>
### Transfers 2016 ~ Loss 2014 + Transfers 2014 + Loss 2015 + Transfers 2015

In [15]:
# INDIVIDUAL GROUP Transfers 2016 = Loss 2014 + Transfers 2014 + Loss 2015 + Transfers 2015

no_nulls = ind_data[['ProfitsPMM_14', 'TransfersPMM_14', 'ProfitsPMM_15',
                     'TransfersPMM_15', 'TransfersPMM']].dropna()
X = no_nulls[['ProfitsPMM_14', 'TransfersPMM_14', 'ProfitsPMM_15', 'TransfersPMM_15']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.792
Method:                 Least Squares   F-statistic:                     275.3
Date:                Wed, 29 May 2019   Prob (F-statistic):           1.46e-96
Time:                        17:27:57   Log-Likelihood:                -1565.9
No. Observations:                 290   AIC:                             3142.
Df Residuals:                     285   BIC:                             3160.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               2.0190      3.532      0.572      0.568      -4.933       8.971
ProfitsPMM_14      -0.1135      0.056     -2.023      0.044      -0.224      -0.003
TransfersPMM_14    -0.0179      0.069     -0.259      0.796      -0.154       0.118
ProfitsPMM_15      -0.0702      0.040     -1.762      0.079      -0.149       0.008
TransfersPMM_15     1.1892      0.053     22.485      0.000       1.085       1.293
==============================================================================
Omnibus:                       47.642   Durbin-Watson:                   2.058
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              383.582
Skew:                           0.300   Prob(JB):                     5.08e-84
Kurtosis:                       8.602   Cond. No.                         123.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
# SMALL GROUP Transfers 2016 = Loss 2014 + Transfers 2014 + Loss 2015 + Transfers 2015

no_nulls = sg_data[['ProfitsPMM_14', 'TransfersPMM_14', 'ProfitsPMM_15',
                     'TransfersPMM_15', 'TransfersPMM']].dropna()
X = no_nulls[['ProfitsPMM_14', 'TransfersPMM_14', 'ProfitsPMM_15', 'TransfersPMM_15']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.476
Model:                            OLS   Adj. R-squared:                  0.471
Method:                 Least Squares   F-statistic:                     96.74
Date:                Wed, 29 May 2019   Prob (F-statistic):           1.69e-58
Time:                        17:27:57   Log-Likelihood:                -2083.7
No. Observations:                 431   AIC:                             4177.
Df Residuals:                     426   BIC:                             4198.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -6.2978      3.270     -1.926      0.055     -12.725       0.130
ProfitsPMM_14      -0.0542      0.037     -1.462      0.144      -0.127       0.019
TransfersPMM_14     0.2689      0.067      4.034      0.000       0.138       0.400
ProfitsPMM_15       0.0874      0.036      2.418      0.016       0.016       0.158
TransfersPMM_15     0.5865      0.058     10.184      0.000       0.473       0.700
==============================================================================
Omnibus:                      304.846   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11036.635
Skew:                          -2.512   Prob(JB):                         0.00
Kurtosis:                      27.276   Cond. No.                         296.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<a id='b4'></a>
### Transfers 2015 ~ Loss 2014 + Transfers 2014

In [17]:
# INDIVIDUAL GROUP Transfers 2015 = Loss 2014 + Transfers 2014

no_nulls = ind_data[['ProfitsPMM_14', 'TransfersPMM_14', 'TransfersPMM_15']].dropna()
X = no_nulls[['ProfitsPMM_14', 'TransfersPMM_14']]
y = no_nulls['TransfersPMM_15']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TransfersPMM_15   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.513
Method:                 Least Squares   F-statistic:                     198.8
Date:                Wed, 29 May 2019   Prob (F-statistic):           1.72e-59
Time:                        17:27:57   Log-Likelihood:                -2079.3
No. Observations:                 376   AIC:                             4165.
Df Residuals:                     373   BIC:                             4176.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -2.4764      3.351     -0.739      0.460      -9.066       4.114
ProfitsPMM_14       0.0220      0.042      0.529      0.597      -0.060       0.104
TransfersPMM_14     0.9135      0.046     19.712      0.000       0.822       1.005
==============================================================================
Omnibus:                       86.041   Durbin-Watson:                   1.932
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1881.247
Skew:                           0.252   Prob(JB):                         0.00
Kurtosis:                      13.947   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
# SMALL GROUP Transfers 2015 = Loss 2014 + Transfers 2014

no_nulls = sg_data[['ProfitsPMM_14', 'TransfersPMM_14', 'TransfersPMM_15']].dropna()
X = no_nulls[['ProfitsPMM_14', 'TransfersPMM_14']]
y = no_nulls['TransfersPMM_15']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TransfersPMM_15   R-squared:                       0.331
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     122.9
Date:                Wed, 29 May 2019   Prob (F-statistic):           4.33e-44
Time:                        17:27:57   Log-Likelihood:                -2458.8
No. Observations:                 499   AIC:                             4924.
Df Residuals:                     496   BIC:                             4936.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -6.7551      2.847     -2.372      0.018     -12.350      -1.161
ProfitsPMM_14       0.0476      0.029      1.617      0.107      -0.010       0.105
TransfersPMM_14     0.7656      0.054     14.096      0.000       0.659       0.872
==============================================================================
Omnibus:                       91.685   Durbin-Watson:                   1.841
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1165.942
Skew:                          -0.323   Prob(JB):                    6.59e-254
Kurtosis:                      10.461   Cond. No.                         178.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<a id='b5'></a>
### Transfers 2016 ~ Loss 2015 + Transfers 2015

In [19]:
# INDIVIDUAL GROUP Transfers 2016 = Loss 2015 + Transfers 2015

no_nulls = ind_data[['ProfitsPMM_15', 'TransfersPMM_15', 'TransfersPMM']].dropna()
X = no_nulls[['ProfitsPMM_15', 'TransfersPMM_15']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.747
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                     530.8
Date:                Wed, 29 May 2019   Prob (F-statistic):          5.90e-108
Time:                        17:27:57   Log-Likelihood:                -1974.3
No. Observations:                 362   AIC:                             3955.
Df Residuals:                     359   BIC:                             3966.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.9639      3.006      0.321      0.749      -4.948       6.876
ProfitsPMM_15      -0.1297      0.033     -3.931      0.000      -0.195      -0.065
TransfersPMM_15     1.1158      0.034     32.412      0.000       1.048       1.184
==============================================================================
Omnibus:                       54.291   Durbin-Watson:                   2.122
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              315.618
Skew:                           0.427   Prob(JB):                     2.91e-69
Kurtosis:                       7.494   Cond. No.                         91.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
# SMALL GROUP Transfers 2016 = Loss 2015 + Transfers 2015

no_nulls = sg_data[['ProfitsPMM_15', 'TransfersPMM_15', 'TransfersPMM']].dropna()
X = no_nulls[['ProfitsPMM_15', 'TransfersPMM_15']]
y = no_nulls['TransfersPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           TransfersPMM   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                     156.5
Date:                Wed, 29 May 2019   Prob (F-statistic):           1.95e-52
Time:                        17:27:57   Log-Likelihood:                -2256.4
No. Observations:                 458   AIC:                             4519.
Df Residuals:                     455   BIC:                             4531.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -8.9256      3.012     -2.963      0.003     -14.845      -3.006
ProfitsPMM_15       0.0565      0.030      1.904      0.058      -0.002       0.115
TransfersPMM_15     0.6236      0.045     13.952      0.000       0.536       0.711
==============================================================================
Omnibus:                      269.886   Durbin-Watson:                   2.079
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10799.031
Skew:                          -1.876   Prob(JB):                         0.00
Kurtosis:                      26.491   Cond. No.                         192.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<a id='b6'></a>
### Costs 2016 = Premiums 2016 + Transfers 2016

In [21]:
# INDIVIDUAL GROUP Costs 2016 = Premiums 2016 + Transfers 2016

no_nulls = ind_data[['PremiumsPMM', 'TransfersPMM', 'CostsPMM']].dropna()
X = no_nulls[['PremiumsPMM', 'TransfersPMM']]
y = no_nulls['CostsPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               CostsPMM   R-squared:                       0.792
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     792.1
Date:                Wed, 29 May 2019   Prob (F-statistic):          9.53e-143
Time:                        17:27:57   Log-Likelihood:                -2551.1
No. Observations:                 420   AIC:                             5108.
Df Residuals:                     417   BIC:                             5120.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           23.4054     27.238      0.859      0.391     -30.135      76.946
PremiumsPMM      1.0120      0.072     14.117      0.000       0.871       1.153
TransfersPMM     1.3490      0.049     27.779      0.000       1.254       1.444
==============================================================================
Omnibus:                      352.898   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            32911.886
Skew:                           2.921   Prob(JB):                         0.00
Kurtosis:                      45.972   Cond. No.                     2.03e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.03e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
# SMALL GROUP Costs 2016 = Premiums 2016 + Transfers 2016

no_nulls = sg_data[['PremiumsPMM', 'TransfersPMM', 'CostsPMM']].dropna()
X = no_nulls[['PremiumsPMM', 'TransfersPMM']]
y = no_nulls['CostsPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               CostsPMM   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.367
Method:                 Least Squares   F-statistic:                     138.5
Date:                Wed, 29 May 2019   Prob (F-statistic):           4.68e-48
Time:                        17:27:57   Log-Likelihood:                -2666.6
No. Observations:                 475   AIC:                             5339.
Df Residuals:                     472   BIC:                             5352.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           54.4131     19.452      2.797      0.005      16.190      92.636
PremiumsPMM      0.6818      0.045     15.161      0.000       0.593       0.770
TransfersPMM     0.1995      0.067      2.979      0.003       0.068       0.331
==============================================================================
Omnibus:                      450.922   Durbin-Watson:                   1.675
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            31653.800
Skew:                           3.805   Prob(JB):                         0.00
Kurtosis:                      42.261   Cond. No.                     2.73e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.73e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<a id='b7'></a>
### Costs 2015 = Premiums 2015 + Transfers 2015

In [23]:
# INDIVIDUAL GROUP Costs 2015 = Premiums 2015 + Transfers 2015

no_nulls = ind_data[['PremiumsPMM_15', 'TransfersPMM_15', 'CostsPMM_15']].dropna()
X = no_nulls[['PremiumsPMM_15', 'TransfersPMM_15']]
y = no_nulls['CostsPMM_15']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            CostsPMM_15   R-squared:                       0.659
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                     434.6
Date:                Wed, 29 May 2019   Prob (F-statistic):          7.91e-106
Time:                        17:27:57   Log-Likelihood:                -2771.1
No. Observations:                 453   AIC:                             5548.
Df Residuals:                     450   BIC:                             5561.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              41.3172     25.373      1.628      0.104      -8.547      91.181
PremiumsPMM_15      0.9894      0.073     13.640      0.000       0.847       1.132
TransfersPMM_15     1.2136      0.066     18.519      0.000       1.085       1.342
==============================================================================
Omnibus:                      232.820   Durbin-Watson:                   1.818
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3636.206
Skew:                           1.822   Prob(JB):                         0.00
Kurtosis:                      16.393   Cond. No.                     1.74e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.74e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
# SMALL GROUP Costs 2015 = Premiums 2015 + Transfers 2015

no_nulls = sg_data[['PremiumsPMM_15', 'TransfersPMM_15', 'CostsPMM_15']].dropna()
X = no_nulls[['PremiumsPMM_15', 'TransfersPMM_15']]
y = no_nulls['CostsPMM_15']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            CostsPMM_15   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.428
Method:                 Least Squares   F-statistic:                     203.4
Date:                Wed, 29 May 2019   Prob (F-statistic):           1.56e-66
Time:                        17:27:57   Log-Likelihood:                -3011.9
No. Observations:                 542   AIC:                             6030.
Df Residuals:                     539   BIC:                             6043.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              31.2007     15.457      2.018      0.044       0.836      61.565
PremiumsPMM_15      0.7186      0.037     19.258      0.000       0.645       0.792
TransfersPMM_15     0.2252      0.060      3.757      0.000       0.107       0.343
==============================================================================
Omnibus:                      249.117   Durbin-Watson:                   1.918
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2478.028
Skew:                           1.756   Prob(JB):                         0.00
Kurtosis:                      12.869   Cond. No.                     2.36e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.36e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<a id='b8'></a>
### Costs 2014 = Premiums 2014 + Transfers 2014

In [25]:
# INDIVIDUAL GROUP Costs 2014 = Premiums 2014 + Transfers 2014

no_nulls = ind_data[['PremiumsPMM_14', 'TransfersPMM_14', 'CostsPMM_14']].dropna()
X = no_nulls[['PremiumsPMM_14', 'TransfersPMM_14']]
y = no_nulls['CostsPMM_14']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            CostsPMM_14   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     375.8
Date:                Wed, 29 May 2019   Prob (F-statistic):           1.42e-92
Time:                        17:27:57   Log-Likelihood:                -2458.2
No. Observations:                 403   AIC:                             4922.
Df Residuals:                     400   BIC:                             4934.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              45.2620     19.717      2.296      0.022       6.500      84.025
PremiumsPMM_14      0.9510      0.061     15.487      0.000       0.830       1.072
TransfersPMM_14     1.4109      0.082     17.164      0.000       1.249       1.572
==============================================================================
Omnibus:                      260.126   Durbin-Watson:                   1.749
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4245.160
Skew:                           2.452   Prob(JB):                         0.00
Kurtosis:                      18.125   Cond. No.                     1.20e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.2e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [26]:
# SMALL GROUP Costs 2014 = Premiums 2014 + Transfers 2014

no_nulls = sg_data[['PremiumsPMM_14', 'TransfersPMM_14', 'CostsPMM_14']].dropna()
X = no_nulls[['PremiumsPMM_14', 'TransfersPMM_14']]
y = no_nulls['CostsPMM_14']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            CostsPMM_14   R-squared:                       0.608
Model:                            OLS   Adj. R-squared:                  0.607
Method:                 Least Squares   F-statistic:                     416.4
Date:                Wed, 29 May 2019   Prob (F-statistic):          7.65e-110
Time:                        17:27:57   Log-Likelihood:                -2858.3
No. Observations:                 539   AIC:                             5723.
Df Residuals:                     536   BIC:                             5736.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              24.5064     10.206      2.401      0.017       4.458      44.555
PremiumsPMM_14      0.7301      0.026     28.301      0.000       0.679       0.781
TransfersPMM_14     0.3895      0.073      5.361      0.000       0.247       0.532
==============================================================================
Omnibus:                       96.586   Durbin-Watson:                   1.550
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              699.507
Skew:                           0.554   Prob(JB):                    1.27e-152
Kurtosis:                       8.470   Cond. No.                     1.92e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.92e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<a id="b9a"></a>
### Premiums - Costs 2016 ~ Transfers 2016

In [27]:
# INDIVIDUAL Profits 2016 ~ Transfers 2016

no_nulls = ind_data[['PremMinusCostsPMM', 'TransfersPMM']].dropna()
X = no_nulls[['TransfersPMM']]
y = no_nulls['PremMinusCostsPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      PremMinusCostsPMM   R-squared:                       0.694
Model:                            OLS   Adj. R-squared:                  0.693
Method:                 Least Squares   F-statistic:                     945.9
Date:                Wed, 29 May 2019   Prob (F-statistic):          2.11e-109
Time:                        17:27:57   Log-Likelihood:                -2551.2
No. Observations:                 420   AIC:                             5106.
Df Residuals:                     418   BIC:                             5114.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -27.8878      5.159     -5.406      0.000     -38.028     -17.748
TransfersPMM    -1.3525      0.044    -30.756      0.000      -1.439      -1.266
==============================================================================
Omnibus:                      352.830   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33094.576
Skew:                          -2.918   Prob(JB):                         0.00
Kurtosis:                      46.094   Cond. No.                         118.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
# SMALL GROUP Profits 2016 ~ Transfers 2016

no_nulls = sg_data[['PremMinusCostsPMM', 'TransfersPMM']].dropna()
X = no_nulls[['TransfersPMM']]
y = no_nulls['PremMinusCostsPMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      PremMinusCostsPMM   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.529
Date:                Wed, 29 May 2019   Prob (F-statistic):              0.217
Time:                        17:27:57   Log-Likelihood:                -2690.6
No. Observations:                 475   AIC:                             5385.
Df Residuals:                     473   BIC:                             5393.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           81.3707      3.330     24.439      0.000      74.828      87.913
TransfersPMM    -0.0844      0.068     -1.237      0.217      -0.218       0.050
==============================================================================
Omnibus:                      427.037   Durbin-Watson:                   1.719
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            28373.213
Skew:                          -3.487   Prob(JB):                         0.00
Kurtosis:                      40.215   Cond. No.                         50.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<a id="b9b"></a>
### Premiums - Costs 2015 ~ Transfers 2015

In [29]:
# INDIVIDUAL Profits 2015 ~ Transfers 2015

no_nulls = ind_data[['PremMinusCostsPMM_15', 'TransfersPMM_15']].dropna()
X = no_nulls[['TransfersPMM_15']]
y = no_nulls['PremMinusCostsPMM_15']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     PremMinusCostsPMM_15   R-squared:                       0.474
Model:                              OLS   Adj. R-squared:                  0.473
Method:                   Least Squares   F-statistic:                     406.3
Date:                  Wed, 29 May 2019   Prob (F-statistic):           6.78e-65
Time:                          17:27:57   Log-Likelihood:                -2771.1
No. Observations:                   453   AIC:                             5546.
Df Residuals:                       451   BIC:                             5554.
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             -37.6902      5.188     -7.265      0.000     -47.886     -27.494
TransfersPMM_15    -1.2097      0.060    -20.157      0.000      -1.328      -1.092
==============================================================================
Omnibus:                      232.316   Durbin-Watson:                   1.817
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3620.005
Skew:                          -1.817   Prob(JB):                         0.00
Kurtosis:                      16.363   Cond. No.                         86.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
# SMALL GROUP Profits 2015 ~ Transfers 2015

no_nulls = sg_data[['PremMinusCostsPMM_15', 'TransfersPMM_15']].dropna()
X = no_nulls[['TransfersPMM_15']]
y = no_nulls['PremMinusCostsPMM_15']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     PremMinusCostsPMM_15   R-squared:                       0.014
Model:                              OLS   Adj. R-squared:                  0.012
Method:                   Least Squares   F-statistic:                     7.713
Date:                  Wed, 29 May 2019   Prob (F-statistic):            0.00567
Time:                          17:27:57   Log-Likelihood:                -3039.1
No. Observations:                   542   AIC:                             6082.
Df Residuals:                       540   BIC:                             6091.
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              83.4579      2.945     28.337      0.000      77.672      89.243
TransfersPMM_15    -0.1737      0.063     -2.777      0.006      -0.297      -0.051
==============================================================================
Omnibus:                      195.042   Durbin-Watson:                   1.858
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1474.574
Skew:                          -1.373   Prob(JB):                         0.00
Kurtosis:                      10.600   Cond. No.                         48.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
np.mean(ind_data["TransfersPMM_14"])

15.676263792541903

<a id="b9c"></a>
### Premiums - Costs 2014 ~ Transfers 2014

In [32]:
# INDIVIDUAL Profits 2014 ~ Transfers 2014

no_nulls = ind_data[['PremMinusCostsPMM_14', 'TransfersPMM_14']].dropna()
X = no_nulls[['TransfersPMM_14']]
y = no_nulls['PremMinusCostsPMM_14']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     PremMinusCostsPMM_14   R-squared:                       0.439
Model:                              OLS   Adj. R-squared:                  0.437
Method:                   Least Squares   F-statistic:                     313.3
Date:                  Wed, 29 May 2019   Prob (F-statistic):           3.22e-52
Time:                          17:27:58   Log-Likelihood:                -2458.5
No. Observations:                   403   AIC:                             4921.
Df Residuals:                       401   BIC:                             4929.
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             -30.1594      5.530     -5.454      0.000     -41.030     -19.288
TransfersPMM_14    -1.3919      0.079    -17.700      0.000      -1.546      -1.237
==============================================================================
Omnibus:                      257.963   Durbin-Watson:                   1.737
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4317.559
Skew:                          -2.413   Prob(JB):                         0.00
Kurtosis:                      18.292   Cond. No.                         72.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [33]:
# INDIVIDUAL Profits 2014 ~ Transfers 2014

no_nulls = ind_data[['PremMinusCostsPMM_14', 'TransfersPMM_14']].dropna()
X = no_nulls[['TransfersPMM_14']]
y = no_nulls['PremMinusCostsPMM_14']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     PremMinusCostsPMM_14   R-squared:                       0.439
Model:                              OLS   Adj. R-squared:                  0.437
Method:                   Least Squares   F-statistic:                     313.3
Date:                  Wed, 29 May 2019   Prob (F-statistic):           3.22e-52
Time:                          17:27:58   Log-Likelihood:                -2458.5
No. Observations:                   403   AIC:                             4921.
Df Residuals:                       401   BIC:                             4929.
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             -30.1594      5.530     -5.454      0.000     -41.030     -19.288
TransfersPMM_14    -1.3919      0.079    -17.700      0.000      -1.546      -1.237
==============================================================================
Omnibus:                      257.963   Durbin-Watson:                   1.737
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4317.559
Skew:                          -2.413   Prob(JB):                         0.00
Kurtosis:                      18.292   Cond. No.                         72.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
# SMALL GROUP Profits 2014 ~ Transfers 2014

no_nulls = sg_data[['PremMinusCostsPMM_14', 'TransfersPMM_14']].dropna()
X = no_nulls[['TransfersPMM_14']]
y = no_nulls['PremMinusCostsPMM_14']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     PremMinusCostsPMM_14   R-squared:                       0.041
Model:                              OLS   Adj. R-squared:                  0.039
Method:                   Least Squares   F-statistic:                     23.01
Date:                  Wed, 29 May 2019   Prob (F-statistic):           2.09e-06
Time:                          17:27:58   Log-Likelihood:                -2908.4
No. Observations:                   539   AIC:                             5821.
Df Residuals:                       537   BIC:                             5829.
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              79.8383      2.365     33.756      0.000      75.192      84.484
TransfersPMM_14    -0.3821      0.080     -4.797      0.000      -0.539      -0.226
==============================================================================
Omnibus:                       38.492   Durbin-Watson:                   1.624
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              140.316
Skew:                           0.177   Prob(JB):                     3.39e-31
Kurtosis:                       5.474   Cond. No.                         30.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<a id="b10a"></a>
### Premiums - Costs + Reinsurance 2016 ~ Transfers 2016

In [35]:
no_nulls = ind_data[['PremMinusCostsPlusReinsurancePMM', 'TransfersPMM']].dropna()
X = no_nulls[['TransfersPMM']]
y = no_nulls['PremMinusCostsPlusReinsurancePMM']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                   
============================================================================================
Dep. Variable:     PremMinusCostsPlusReinsurancePMM   R-squared:                       0.676
Model:                                          OLS   Adj. R-squared:                  0.675
Method:                               Least Squares   F-statistic:                     872.6
Date:                              Wed, 29 May 2019   Prob (F-statistic):          2.21e-104
Time:                                      17:27:58   Log-Likelihood:                -2513.3
No. Observations:                               420   AIC:                             5031.
Df Residuals:                                   418   BIC:                             5039.
Df Model:                                         1                                         
Covariance Type:                          nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -5.1309      4.714     -1.088      0.277     -14.397       4.135
TransfersPMM    -1.1870      0.040    -29.540      0.000      -1.266      -1.108
==============================================================================
Omnibus:                      349.981   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            29428.189
Skew:                          -2.917   Prob(JB):                         0.00
Kurtosis:                      43.590   Cond. No.                         118.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<a id="b10b"></a>
### Premiums - Costs + Reinsurance 2015 ~ Transfers 2015

In [36]:
no_nulls = ind_data[['PremMinusCostsPlusReinsurancePMM_15', 'TransfersPMM_15']].dropna()
X = no_nulls[['TransfersPMM_15']]
y = no_nulls['PremMinusCostsPlusReinsurancePMM_15']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     OLS Regression Results                                    
===============================================================================================
Dep. Variable:     PremMinusCostsPlusReinsurancePMM_15   R-squared:                       0.441
Model:                                             OLS   Adj. R-squared:                  0.440
Method:                                  Least Squares   F-statistic:                     355.7
Date:                                 Wed, 29 May 2019   Prob (F-statistic):           6.35e-59
Time:                                         17:27:58   Log-Likelihood:                -2693.3
No. Observations:                                  453   AIC:                             5391.
Df Residuals:                                      451   BIC:                             5399.
Df Model:                                            1                                         
Covariance Type:                             nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               3.4335      4.369      0.786      0.432      -5.153      12.020
TransfersPMM_15    -0.9533      0.051    -18.860      0.000      -1.053      -0.854
==============================================================================
Omnibus:                      219.813   Durbin-Watson:                   1.800
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3043.332
Skew:                          -1.724   Prob(JB):                         0.00
Kurtosis:                      15.221   Cond. No.                         86.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<a id="b10c"></a>
### Premiums - Costs + Reinsurance 2014 ~ Transfers 2014

In [37]:
no_nulls = ind_data[['PremMinusCostsPlusReinsurancePMM_14', 'TransfersPMM_14']].dropna()
X = no_nulls[['TransfersPMM_14']]
y = no_nulls['PremMinusCostsPlusReinsurancePMM_14']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     OLS Regression Results                                    
===============================================================================================
Dep. Variable:     PremMinusCostsPlusReinsurancePMM_14   R-squared:                       0.364
Model:                                             OLS   Adj. R-squared:                  0.363
Method:                                  Least Squares   F-statistic:                     229.8
Date:                                 Wed, 29 May 2019   Prob (F-statistic):           2.33e-41
Time:                                         17:27:58   Log-Likelihood:                -2351.4
No. Observations:                                  403   AIC:                             4707.
Df Residuals:                                      401   BIC:                             4715.
Df Model:                                            1                                         
Covariance Type:                             nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              21.6205      4.239      5.100      0.000      13.286      29.955
TransfersPMM_14    -0.9139      0.060    -15.160      0.000      -1.032      -0.795
==============================================================================
Omnibus:                      185.996   Durbin-Watson:                   1.682
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2026.924
Skew:                          -1.665   Prob(JB):                         0.00
Kurtosis:                      13.470   Cond. No.                         72.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<a id='c1'></a>
## Effect of losses in 2014 and 2015 on exiting the market in 2016

In [38]:
# Looking at the effect of losses in 2014 and 2015 on exiting the market in 2016

ind_data_exit = ind_data_14.merge(ind_data_15, on='HIOS ID', how='left', suffixes=('_14', '_15'))\
.merge(ind_data_16, on='HIOS ID', how='left')
ind_data_exit["LossQuintile"] = pd.qcut(ind_data_exit['ProfitsPMM_14'], 5, labels=False)
ind_data_exit["ExitedMarketIn2016"] = np.where(ind_data_exit[['TransfersPMM']].isnull(), 1, 0)
ind_data_exit = pd.concat([ind_data_exit, pd.get_dummies(ind_data_exit['LossQuintile'])], axis=1)
ind_data_exit.columns = ind_data_exit.columns.astype(str)

In [39]:
# Looking at the effect of losses in 2014 and 2015 on exiting the market in 2016

sg_data_exit = sg_data_14.merge(sg_data_15, on='HIOS ID', how='left', suffixes=('_14', '_15'))\
.merge(sg_data_16, on='HIOS ID', how='left')
sg_data_exit["LossQuintile"] = pd.qcut(sg_data_exit['ProfitsPMM_14'], 5, labels=False)
sg_data_exit["ExitedMarketIn2016"] = np.where(sg_data_exit[['TransfersPMM']].isnull(), 1, 0)
sg_data_exit = pd.concat([sg_data_exit, pd.get_dummies(sg_data_exit['LossQuintile'])], axis=1)
sg_data_exit.columns = sg_data_exit.columns.astype(str)

<a id='c2'></a>
### ExitedMarketIn2016 = Loss 2014 Q1 + Loss 2014 Q5

In [40]:
# ExitedMarketIn 2016 = Loss 2014 Q1 + Loss 2014 Q5

# "0" = you are losing money (loss is negative, which means Costs > Premiums + Transfers + Reinsurance)
# "4" = you are making money (loss is positive, which means Costs < Premiums + Transfers + Reinsurance)

no_nulls = ind_data_exit[['0', '4', 'ExitedMarketIn2016']].dropna()
X = no_nulls[['0', '4']]
y = no_nulls['ExitedMarketIn2016']
X = sm.add_constant(X)
est = sm.Logit(y, X).fit()
est.summary()

Optimization terminated successfully.
         Current function value: 0.578562
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     ExitedMarketIn2016   No. Observations:                  403
Model:                          Logit   Df Residuals:                      400
Method:                           MLE   Df Model:                            2
Date:                Wed, 29 May 2019   Pseudo R-squ.:                 0.01299
Time:                        17:27:58   Log-Likelihood:                -233.16
converged:                       True   LL-Null:                       -236.23
                                        LLR p-value:                   0.04654
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9134      0.143     -6.410      0.000      -1.193      -0.634
0              0.2202      0.275      0.800      0.424      -0.320       0.760
4             -0.6522      0.327     -1.997      0.046      -1.292      -0.012
==============================================================================
"""

In [41]:
# ExitedMarketIn 2016 = Loss 2014 Q1 + Loss 2014 Q5

# "0" = you are losing money (loss is negative, which means Costs > Premiums + Transfers + Reinsurance)
# "4" = you are making money (loss is positive, which means Costs < Premiums + Transfers + Reinsurance)

no_nulls = sg_data_exit[['0', '4', 'ExitedMarketIn2016']].dropna()
X = no_nulls[['0', '4']]
y = no_nulls['ExitedMarketIn2016']
X = sm.add_constant(X)
est = sm.Logit(y, X).fit()
est.summary()

Optimization terminated successfully.
         Current function value: 0.469176
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     ExitedMarketIn2016   No. Observations:                  539
Model:                          Logit   Df Residuals:                      536
Method:                           MLE   Df Model:                            2
Date:                Wed, 29 May 2019   Pseudo R-squ.:                 0.05852
Time:                        17:27:58   Log-Likelihood:                -252.89
converged:                       True   LL-Null:                       -268.60
                                        LLR p-value:                 1.491e-07
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.9566      0.169    -11.583      0.000      -2.288      -1.625
0              1.4259      0.261      5.459      0.000       0.914       1.938
4              0.8580      0.279      3.074      0.002       0.311       1.405
==============================================================================
"""

<a id='c3'></a>
### ExitedMarketIn2016 = Profits 2014 + Profits 2015

In [42]:
# ExitedMarketIn 2016 = Loss 2014 + Loss 2015

no_nulls = ind_data_exit[['ProfitsPMM_14', 'ProfitsPMM_15', 'ExitedMarketIn2016']].dropna()
print("Individual Group")
print("Loss 14 Avg: %f, Range: %f" % (no_nulls["ProfitsPMM_14"].mean(), no_nulls["ProfitsPMM_14"].max() - no_nulls["ProfitsPMM_14"].min()))
print("Loss 15 Avg: %f, Range: %f" % (no_nulls["ProfitsPMM_15"].mean(), no_nulls["ProfitsPMM_15"].max() - no_nulls["ProfitsPMM_15"].min()))
no_nulls["ProfitsPMM_14_Norm"] = (no_nulls["ProfitsPMM_14"] - no_nulls["ProfitsPMM_14"].mean()) / (no_nulls["ProfitsPMM_14"].max() - no_nulls["ProfitsPMM_14"].min())
no_nulls["ProfitsPMM_15_Norm"] = (no_nulls["ProfitsPMM_15"] - no_nulls["ProfitsPMM_15"].mean()) / (no_nulls["ProfitsPMM_15"].max() - no_nulls["ProfitsPMM_15"].min())
X = no_nulls[['ProfitsPMM_14_Norm', 'ProfitsPMM_15_Norm']]
y = no_nulls['ExitedMarketIn2016']
X = sm.add_constant(X)
est = sm.Logit(y, X).fit()
est.summary()

Individual Group
Loss 14 Avg: 22.879370, Range: 677.705621
Loss 15 Avg: 4.691754, Range: 1141.735553
Optimization terminated successfully.
         Current function value: 0.522292
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     ExitedMarketIn2016   No. Observations:                  376
Model:                          Logit   Df Residuals:                      373
Method:                           MLE   Df Model:                            2
Date:                Wed, 29 May 2019   Pseudo R-squ.:                 0.02871
Time:                        17:27:58   Log-Likelihood:                -196.38
converged:                       True   LL-Null:                       -202.19
                                        LLR p-value:                  0.003015
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -1.2515      0.126     -9.909      0.000      -1.499      -1.004
ProfitsPMM_14_Norm    -3.6449      1.262     -2.888      0.004      -6.118      -1.171
ProfitsPMM_15_Norm     0.2177      1.692      0.129      0.898      -3.099       3.535
======================================================================================
"""

In [43]:
# ExitedMarketIn 2016 = Loss 2014 + Loss 2015

no_nulls = sg_data_exit[['ProfitsPMM_14', 'ProfitsPMM_15', 'ExitedMarketIn2016']].dropna()
print("Small Group")
print("Loss 14 Avg: %f, Range: %f" % (no_nulls["ProfitsPMM_14"].mean(), no_nulls["ProfitsPMM_14"].max() - no_nulls["ProfitsPMM_14"].min()))
print("Loss 15 Avg: %f, Range: %f" % (no_nulls["ProfitsPMM_15"].mean(), no_nulls["ProfitsPMM_15"].max() - no_nulls["ProfitsPMM_15"].min()))
no_nulls["ProfitsPMM_14_Norm"] = (no_nulls["ProfitsPMM_14"] - no_nulls["ProfitsPMM_14"].mean()) / (no_nulls["ProfitsPMM_14"].max() - no_nulls["ProfitsPMM_14"].min())
no_nulls["ProfitsPMM_15_Norm"] = (no_nulls["ProfitsPMM_15"] - no_nulls["ProfitsPMM_15"].mean()) / (no_nulls["ProfitsPMM_15"].max() - no_nulls["ProfitsPMM_15"].min())
X = no_nulls[['ProfitsPMM_14_Norm', 'ProfitsPMM_15_Norm']]
y = no_nulls['ExitedMarketIn2016']
X = sm.add_constant(X)
est = sm.Logit(y, X).fit()
est.summary()

Small Group
Loss 14 Avg: 76.585056, Range: 518.041756
Loss 15 Avg: 78.535117, Range: 708.086313
Optimization terminated successfully.
         Current function value: 0.389014
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     ExitedMarketIn2016   No. Observations:                  499
Model:                          Logit   Df Residuals:                      496
Method:                           MLE   Df Model:                            2
Date:                Wed, 29 May 2019   Pseudo R-squ.:                 0.02292
Time:                        17:27:58   Log-Likelihood:                -194.12
converged:                       True   LL-Null:                       -198.67
                                        LLR p-value:                   0.01053
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -1.8998      0.136    -13.950      0.000      -2.167      -1.633
ProfitsPMM_14_Norm    -2.7060      1.584     -1.708      0.088      -5.811       0.399
ProfitsPMM_15_Norm    -1.5096      1.468     -1.028      0.304      -4.386       1.367
======================================================================================
"""